
# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio for AI-Assisted SQL + ML

**Date:** 2025-10-16  
**Objective:** Build and refine a complete ML pipeline for churn prediction using BigQuery — but with **Gemini-style prompts** guiding SQL generation.

You'll learn to:
- Frame SQL goals as clear prompts
- Generate, test, and debug queries with an AI assistant
- Reflect on each modeling step and your prompt design



## Task 0: Connect to BigQuery

**🎯 Goal:** Verify BigQuery access from Colab.  
**📌 Requirements:** Use `%%bigquery`, get current date and user session.

---

### 🧠 Prompt Template  
> Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab.

---

### 👩‍🏫 Example Prompt  
> Write a SQL query using BigQuery syntax that returns today’s date and the current session user.

---

### ✅ Expected SQL Output
```sql
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;
```

---

### 🔍 Checkpoint  
Query should return a single row with today's date and your user.


In [1]:
# Set your Google Cloud project ID
project_id = 'mgmt-467-47888-471119'

In [2]:
%%bigquery --project {project_id}
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-26,armanhyder32@gmail.com



## Task 1: Prepare ML Table

**🎯 Goal:** Create a clean features table for modeling churn.  
**📌 Requirements:** Use cleaned_features as source, select relevant columns, filter rows with churn_label IS NOT NULL.

---

### 🧠 Prompt Template  
> Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL.

---

### 👩‍🏫 Example Prompt  
> Create a BigQuery table named churn_features from cleaned_features with selected features and where churn_label IS NOT NULL.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;
```

---

### 🔍 Checkpoint  
Table should appear in BigQuery and contain non-null labels.


In [30]:
%%bigquery --project {project_id}
SELECT column_name, data_type
FROM `netflix`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'cleaned_features';

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,user_id,STRING
1,age,FLOAT64
2,country,STRING
3,subscription_plan,STRING
4,total_minutes,FLOAT64
5,avg_rating,FLOAT64
6,search_count,INT64
7,avg_results_per_search,FLOAT64
8,recommendation_click_rate,FLOAT64
9,churn_label,BOOL


In [4]:
%%bigquery --project {project_id}
SELECT column_name, data_type
FROM `netflix`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'watch_history';

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,session_id,STRING
1,user_id,STRING
2,movie_id,STRING
3,watch_date,DATE
4,device_type,STRING
5,watch_duration_minutes,FLOAT64
6,progress_percentage,FLOAT64
7,action,STRING
8,quality,STRING
9,location_country,STRING


In [12]:
%%bigquery --project {project_id}
SELECT column_name, data_type
FROM `netflix`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'recommendation_logs';

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,recommendation_id,STRING
1,user_id,STRING
2,movie_id,STRING
3,recommendation_date,DATE
4,recommendation_type,STRING
5,recommendation_score,FLOAT64
6,was_clicked,BOOL
7,position_in_list,INT64
8,device_type,STRING
9,time_of_day,STRING


In [8]:
%%bigquery --project {project_id}
SELECT column_name, data_type
FROM `netflix`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'search_logs';

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,search_id,STRING
1,user_id,STRING
2,search_query,STRING
3,search_date,DATE
4,results_returned,INT64
5,clicked_result_position,INT64
6,device_type,STRING
7,search_duration_seconds,FLOAT64
8,had_typo,BOOL
9,used_filters,BOOL


In [13]:
%%bigquery --project {project_id}
CREATE OR REPLACE TABLE `netflix.churn_features` AS
SELECT
    u.user_id,
    u.country,
    u.subscription_plan,
    u.age,
    AVG(r.rating) AS avg_rating,
    SUM(wh.watch_duration_minutes) AS total_minutes,
    COUNT(s.search_id) AS search_count,
    AVG(s.results_returned) AS avg_results_per_search,
    AVG(CASE WHEN rl.was_clicked IS TRUE THEN 1 ELSE 0 END) AS recommendation_click_rate,
    CASE
        WHEN MAX(wh.watch_date) IS NULL OR MAX(wh.watch_date) < CURRENT_DATE() - INTERVAL 30 DAY THEN TRUE
        ELSE FALSE
    END AS churn_label
FROM
    `netflix.users` u
LEFT JOIN
    `netflix.watch_history` wh ON u.user_id = wh.user_id
LEFT JOIN
    `netflix.reviews` r ON u.user_id = r.user_id
LEFT JOIN
    `netflix.search_logs` s ON u.user_id = s.user_id
LEFT JOIN
    `netflix.recommendation_logs` rl ON u.user_id = rl.user_id
GROUP BY
    u.user_id, u.country, u.subscription_plan, u.age;

Query is running:   0%|          |

""



## Task 2: Train Logistic Regression Model

**🎯 Goal:** Train a basic BQML logistic regression model.  
**📌 Requirements:** Use churn_features table, predict churn_label from features.

---

### 🧠 Prompt Template  
> Write a CREATE MODEL SQL for logistic regression using churn_label as label and [features] as inputs.

---

### 👩‍🏫 Example Prompt  
> Train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `your_dataset.churn_features`;
```

---

### 🔍 Checkpoint  
Model appears in BigQuery under Models. Training completes.


In [14]:
%%bigquery --project {project_id}
CREATE OR REPLACE MODEL `netflix.churn_model`
OPTIONS(model_type='logistic_reg', input_label_cols=['churn_label']) AS
SELECT
    country,
    subscription_plan,
    age,
    avg_rating,
    total_minutes,
    search_count,
    avg_results_per_search,
    recommendation_click_rate,
    churn_label
FROM
    `netflix.churn_features`;

Query is running:   0%|          |

""



## Task 3: Evaluate Model

**🎯 Goal:** Evaluate the logistic regression model.  
**📌 Requirements:** Use ML.EVALUATE.

---

### 🧠 Prompt Template  
> Write a query to evaluate my logistic regression model using ML.EVALUATE.

---

### 👩‍🏫 Example Prompt  
> Evaluate the churn_model using ML.EVALUATE to get accuracy, precision, recall.

---

### ✅ Expected SQL Output
```sql
SELECT * FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);
```

---

### 🔍 Checkpoint  
View performance metrics: accuracy, log_loss, precision, recall.


In [15]:
%%bigquery --project {project_id}
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.739504,0.0,0.572986,0.519771



## Task 4: Predict Churn

**🎯 Goal:** Use ML.PREDICT to generate churn predictions.  
**📌 Requirements:** Apply model to same input table.

---

### 🧠 Prompt Template  
> Generate SQL to use ML.PREDICT on churn_model and return predictions by user_id.

---

### 👩‍🏫 Example Prompt  
> Predict churn using churn_model. Include user_id, predicted_churn_label, and prediction probability.

---

### ✅ Expected SQL Output
```sql
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
      (SELECT * FROM `your_dataset.churn_features`));
```

---

### 🔍 Checkpoint  
Inspect top churn risk users. Validate probabilities.


In [16]:
%%bigquery --project {project_id}
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `netflix.churn_model`,
      (SELECT * FROM `netflix.churn_features`));

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,predicted_churn_label,predicted_churn_label_probs
0,user_02626,False,"[{'label': True, 'prob': 0.26458161518730733},..."
1,user_00357,False,"[{'label': True, 'prob': 0.2719909876335588}, ..."
2,user_00977,False,"[{'label': True, 'prob': 0.2697646626203773}, ..."
3,user_00317,False,"[{'label': True, 'prob': 0.2724477690304694}, ..."
4,user_01540,False,"[{'label': True, 'prob': 0.27595435224617704},..."
...,...,...,...
9995,user_07233,False,"[{'label': True, 'prob': 0.18241784097173283},..."
9996,user_01599,False,"[{'label': True, 'prob': 0.17687072568395726},..."
9997,user_09078,False,"[{'label': True, 'prob': 0.18269962140861737},..."
9998,user_00327,False,"[{'label': True, 'prob': 0.1651991476075271}, ..."


No charts were generated by quickchart



## Task 5.0: Bucket a Continuous Feature

**🎯 Goal:** Group 'total_minutes' into categories: low, medium, high.  
**📌 Requirements:** Use CASE WHEN or IF statements to create 'watch_time_bucket'.

---

### 🧠 Prompt Template  
> Write SQL that creates a new column watch_time_bucket based on total_minutes thresholds (<100, 100–300, >300).

---

### 👩‍🏫 Example Prompt  
> Create a new column watch_time_bucket with values 'low', 'medium', or 'high' based on total_minutes.

---

### 🔍 Exploration  
How does churn rate vary across these buckets?


In [17]:
%%bigquery --project {project_id}
SELECT
    *,
    CASE
        WHEN total_minutes < 100 THEN 'low'
        WHEN total_minutes BETWEEN 100 AND 300 THEN 'medium'
        WHEN total_minutes > 300 THEN 'high'
        ELSE 'unknown' # Handle potential NULLs or other cases
    END AS watch_time_bucket
FROM
    `netflix.churn_features`;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,country,subscription_plan,age,avg_rating,total_minutes,search_count,avg_results_per_search,recommendation_click_rate,churn_label,watch_time_bucket
0,user_02626,USA,Basic,15.0,2.000000,2048.4,0,NaN,0.000000,False,high
1,user_00357,USA,Basic,29.0,4.000000,1961.2,0,NaN,0.250000,False,high
2,user_00977,USA,Basic,NaN,NaN,2946.4,0,NaN,0.000000,False,high
3,user_00317,Canada,Basic,37.0,4.000000,11824.0,0,NaN,0.600000,False,high
4,user_01540,USA,Basic,47.0,3.000000,6252.0,0,NaN,0.000000,False,high
...,...,...,...,...,...,...,...,...,...,...,...
9995,user_07233,USA,Standard,19.0,4.500000,117096.0,2520,48.428571,0.200000,False,high
9996,user_01599,USA,Standard,41.0,3.750000,170912.0,2688,50.142857,0.250000,True,high
9997,user_09078,USA,Standard,36.0,3.250000,116121.6,2880,42.166667,0.125000,False,high
9998,user_00327,Canada,Standard,10.0,3.833333,101088.0,3024,63.111111,0.125000,False,high



## Task 5.1: Create a Binary Flag Feature

**🎯 Goal:** Add a binary column flag_binge (1 if total_minutes > 500).  
**📌 Requirements:** Use IF logic to create a binary column in SQL.

---

### 🧠 Prompt Template  
> Write a SQL query that adds flag_binge = 1 if total_minutes > 500, else 0.

---

### 👩‍🏫 Example Prompt  
> Add a binary column flag_binge to identify binge-watchers.

---

### 🔍 Exploration  
Are binge-watchers more or less likely to churn?


In [18]:
%%bigquery --project {project_id}
SELECT
    *,
    CASE
        WHEN total_minutes > 500 THEN 1
        ELSE 0
    END AS flag_binge
FROM
    `netflix.churn_features`;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,country,subscription_plan,age,avg_rating,total_minutes,search_count,avg_results_per_search,recommendation_click_rate,churn_label,flag_binge
0,user_02626,USA,Basic,15.0,2.000000,2048.4,0,NaN,0.000000,False,1
1,user_00357,USA,Basic,29.0,4.000000,1961.2,0,NaN,0.250000,False,1
2,user_00977,USA,Basic,NaN,NaN,2946.4,0,NaN,0.000000,False,1
3,user_00317,Canada,Basic,37.0,4.000000,11824.0,0,NaN,0.600000,False,1
4,user_01540,USA,Basic,47.0,3.000000,6252.0,0,NaN,0.000000,False,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,user_07233,USA,Standard,19.0,4.500000,117096.0,2520,48.428571,0.200000,False,1
9996,user_01599,USA,Standard,41.0,3.750000,170912.0,2688,50.142857,0.250000,True,1
9997,user_09078,USA,Standard,36.0,3.250000,116121.6,2880,42.166667,0.125000,False,1
9998,user_00327,Canada,Standard,10.0,3.833333,101088.0,3024,63.111111,0.125000,False,1



## Task 5.2: Create an Interaction Term

**🎯 Goal:** Create plan_region_combo by combining plan_tier and region.  
**📌 Requirements:** Use CONCAT or STRING functions.

---

### 🧠 Prompt Template  
> Generate SQL to create a new column by combining plan_tier and region with an underscore.

---

### 👩‍🏫 Example Prompt  
> Create a column called plan_region_combo as CONCAT(plan_tier, '_', region).

---

### 🔍 Exploration  
Which plan-region combos have highest churn?


In [19]:
%%bigquery --project {project_id}
SELECT
    *,
    CONCAT(subscription_plan, '_', country) AS plan_region_combo
FROM
    `netflix.churn_features`;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,country,subscription_plan,age,avg_rating,total_minutes,search_count,avg_results_per_search,recommendation_click_rate,churn_label,plan_region_combo
0,user_02626,USA,Basic,15.0,2.000000,2048.4,0,NaN,0.000000,False,Basic_USA
1,user_00357,USA,Basic,29.0,4.000000,1961.2,0,NaN,0.250000,False,Basic_USA
2,user_00977,USA,Basic,NaN,NaN,2946.4,0,NaN,0.000000,False,Basic_USA
3,user_00317,Canada,Basic,37.0,4.000000,11824.0,0,NaN,0.600000,False,Basic_Canada
4,user_01540,USA,Basic,47.0,3.000000,6252.0,0,NaN,0.000000,False,Basic_USA
...,...,...,...,...,...,...,...,...,...,...,...
9995,user_07233,USA,Standard,19.0,4.500000,117096.0,2520,48.428571,0.200000,False,Standard_USA
9996,user_01599,USA,Standard,41.0,3.750000,170912.0,2688,50.142857,0.250000,True,Standard_USA
9997,user_09078,USA,Standard,36.0,3.250000,116121.6,2880,42.166667,0.125000,False,Standard_USA
9998,user_00327,Canada,Standard,10.0,3.833333,101088.0,3024,63.111111,0.125000,False,Standard_Canada



## Task 5.3: Add Missingness Indicator Flags

**🎯 Goal:** Add binary flags to capture NULL values in age_band and avg_rating.  
**📌 Requirements:** Use IS NULL logic to create new flag columns.

---

### 🧠 Prompt Template  
> Create a new column is_missing_[col_name] that is 1 when column is NULL, else 0.

---

### 👩‍🏫 Example Prompt  
> Add is_missing_age that flags rows where age_band IS NULL.

---

### 🔍 Exploration  
Do missing values correlate with churn?


In [20]:
%%bigquery --project {project_id}
SELECT
    *,
    CASE WHEN age IS NULL THEN 1 ELSE 0 END AS is_missing_age,
    CASE WHEN avg_rating IS NULL THEN 1 ELSE 0 END AS is_missing_avg_rating,
    CASE WHEN avg_results_per_search IS NULL THEN 1 ELSE 0 END AS is_missing_avg_results_per_search
FROM
    `netflix.churn_features`;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,country,subscription_plan,age,avg_rating,total_minutes,search_count,avg_results_per_search,recommendation_click_rate,churn_label,is_missing_age,is_missing_avg_rating,is_missing_avg_results_per_search
0,user_02626,USA,Basic,15.0,2.000000,2048.4,0,NaN,0.000000,False,0,0,1
1,user_00357,USA,Basic,29.0,4.000000,1961.2,0,NaN,0.250000,False,0,0,1
2,user_00977,USA,Basic,NaN,NaN,2946.4,0,NaN,0.000000,False,1,1,1
3,user_00317,Canada,Basic,37.0,4.000000,11824.0,0,NaN,0.600000,False,0,0,1
4,user_01540,USA,Basic,47.0,3.000000,6252.0,0,NaN,0.000000,False,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,user_07233,USA,Standard,19.0,4.500000,117096.0,2520,48.428571,0.200000,False,0,0,0
9996,user_01599,USA,Standard,41.0,3.750000,170912.0,2688,50.142857,0.250000,True,0,0,0
9997,user_09078,USA,Standard,36.0,3.250000,116121.6,2880,42.166667,0.125000,False,0,0,0
9998,user_00327,Canada,Standard,10.0,3.833333,101088.0,3024,63.111111,0.125000,False,0,0,0



## Task 5.4: Create Time-Based Features (Optional)

**🎯 Goal:** Add a column days_since_last_login.  
**📌 Requirements:** Use DATE_DIFF with CURRENT_DATE and last_login_date.

---

### 🧠 Prompt Template  
> Write SQL to create a column showing days since last login using DATE_DIFF.

---

### 👩‍🏫 Example Prompt  
> Add a column days_since_last_login = DATE_DIFF(CURRENT_DATE(), last_login_date, DAY).

---

### 🔍 Exploration  
Does login recency affect churn rate?



## Task 5.5: Assemble Enhanced Feature Table

**🎯 Goal:** Create churn_features_enhanced with all engineered columns.  
**📌 Requirements:** Include all prior features + engineered columns.

---

### 🧠 Prompt Template  
> Generate SQL to create churn_features_enhanced with new columns: watch_time_bucket, plan_region_combo, flag_binge, etc.

---

### 👩‍🏫 Example Prompt  
> Build a new table churn_features_enhanced with all original features + engineered ones.

---

### 🔍 Exploration  
Are row counts stable? Any NULLs introduced?


In [21]:
%%bigquery --project {project_id}
CREATE OR REPLACE TABLE `netflix.churn_features_enhanced` AS
SELECT
    *,
    CASE
        WHEN total_minutes < 100 THEN 'low'
        WHEN total_minutes BETWEEN 100 AND 300 THEN 'medium'
        WHEN total_minutes > 300 THEN 'high'
        ELSE 'unknown' # Handle potential NULLs or other cases
    END AS watch_time_bucket,
    CASE
        WHEN total_minutes > 500 THEN 1
        ELSE 0
    END AS flag_binge,
    CONCAT(subscription_plan, '_', country) AS plan_region_combo,
    CASE WHEN age IS NULL THEN 1 ELSE 0 END AS is_missing_age,
    CASE WHEN avg_rating IS NULL THEN 1 ELSE 0 END AS is_missing_avg_rating,
    CASE WHEN avg_results_per_search IS NULL THEN 1 ELSE 0 END AS is_missing_avg_results_per_search
FROM
    `netflix.churn_features`;

Query is running:   0%|          |

""


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

In [25]:
%%bigquery --project {project_id}
SELECT COUNT(*) AS row_count
FROM `netflix.churn_features_enhanced`;

Query is running:   0%|          |

Downloading:   0%|          |

,row_count
0,10000


In [26]:
%%bigquery --project {project_id}
SELECT
    COUNTIF(age IS NULL) AS null_age_count,
    COUNTIF(avg_rating IS NULL) AS null_avg_rating_count,
    COUNTIF(total_minutes IS NULL) AS null_total_minutes_count,
    COUNTIF(search_count IS NULL) AS null_search_count,
    COUNTIF(avg_results_per_search IS NULL) AS null_avg_results_per_search_count,
    COUNTIF(recommendation_click_rate IS NULL) AS null_recommendation_click_rate_count,
    COUNTIF(watch_time_bucket IS NULL) AS null_watch_time_bucket_count,
    COUNTIF(flag_binge IS NULL) AS null_flag_binge_count,
    COUNTIF(plan_region_combo IS NULL) AS null_plan_region_combo_count,
    COUNTIF(is_missing_age IS NULL) AS null_is_missing_age_count,
    COUNTIF(is_missing_avg_rating IS NULL) AS null_is_missing_avg_rating_count,
    COUNTIF(is_missing_avg_results_per_search IS NULL) AS null_is_missing_avg_results_per_search_count,
    COUNTIF(churn_label IS NULL) AS null_churn_label_count
FROM `netflix.churn_features_enhanced`;

Query is running:   0%|          |

Downloading:   0%|          |

,null_age_count,null_avg_rating_count,null_total_minutes_count,null_search_count,null_avg_results_per_search_count,null_recommendation_click_rate_count,null_watch_time_bucket_count,null_flag_binge_count,null_plan_region_combo_count,null_is_missing_age_count,null_is_missing_avg_rating_count,null_is_missing_avg_results_per_search_count,null_churn_label_count
0,1194,2239,0,0,814,0,0,0,0,0,0,0,0



## Task 6: Retrain Model on Engineered Features

**🎯 Goal:** Train a logistic regression model using churn_features_enhanced.  
**📌 Requirements:** Use BQML logistic_reg model with new feature columns.

---

### 🧠 Prompt Template  
> Write CREATE MODEL SQL using enhanced features including flags and buckets.

---

### 👩‍🏫 Example Prompt  
> Retrain churn_model_enhanced using watch_time_bucket, flag_binge, plan_region_combo.

---

### 🔍 Exploration  
Does model accuracy improve?


In [22]:
%%bigquery --project {project_id}
CREATE OR REPLACE MODEL `netflix.churn_model_enhanced`
OPTIONS(model_type='logistic_reg', input_label_cols=['churn_label']) AS
SELECT
    country,
    subscription_plan,
    age,
    avg_rating,
    total_minutes,
    search_count,
    avg_results_per_search,
    recommendation_click_rate,
    watch_time_bucket,
    flag_binge,
    plan_region_combo,
    is_missing_age,
    is_missing_avg_rating,
    is_missing_avg_results_per_search,
    churn_label
FROM
    `netflix.churn_features_enhanced`;

Query is running:   0%|          |

""


In [23]:
%%bigquery --project {project_id}
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.739504,0.0,0.572986,0.519771


In [24]:
%%bigquery --project {project_id}
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model_enhanced`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1.0,0.001965,0.741081,0.003922,0.572747,0.515422



## Task 7: Compare Model Performance

**🎯 Goal:** Compare base model vs enhanced model using ML.EVALUATE.  
**📌 Requirements:** Use same evaluation query for both models.

---

### 🧠 Prompt Template  
> Write a SQL query to evaluate churn_model_enhanced and compare with churn_model.

---

### 👩‍🏫 Example Prompt  
> Compare ML.EVALUATE output from both models side-by-side.

---

### 🔍 Exploration  
Which features made the most difference?


While the `ML.EVALUATE` output provides overall model performance metrics (such as accuracy, precision, recall, and ROC AUC), it does not directly indicate the importance of individual features. However, by comparing the evaluation results of the original model (trained on basic features) and the enhanced model (trained with engineered features like watch time buckets, binge flags, plan-region combinations, and missingness indicators), we can infer the overall impact of the added features.

If the enhanced model shows improved performance metrics compared to the original model, it suggests that the **set of engineered features** collectively provided valuable information that helped the model better predict churn. These new features likely captured nuances and patterns in the data that the original features alone did not, leading to a more robust or accurate model.

To determine the specific contribution of each individual engineered feature, a dedicated feature importance analysis method would be required, which is beyond the scope of the current `ML.EVALUATE` function.